# User Days 2020 NWB Tutorial - Optical Physiology

## Introduction
In this tutorial, we will create fake data for a hypothetical optical physiology experiment with a freely moving animal. The types of data we will convert are:
* Subject (species, strain, age, etc.)
* Animal position
* Trials
* Acquired two-photon images
* Image segmentation (ROIs)
* Fluorescence and dF/F response

## Installing PyNWB
First, install PyNWB using pip or conda. You will need Python 3.5+ installed.
- `pip install pynwb`
- `conda install -c conda-forge pynwb`

## Set up the NWB file
An NWB file represents a single session of an experiment. Each file must have a session description, identifier, and session start time. Create a new `NWBFile` object with those and additional metadata. For all PyNWB constructors, we recommend using keyword arguments.

In [1]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwbfile = NWBFile(
    session_description='Head-fixed mouse on a wheel',
    identifier='Mouse5_Day3',
    session_start_time=start_time,
    session_id='session_1234',                                # optional
    experimenter='My Name',                                   # optional
    lab='My Lab Name',                                        # optional
    institution='University of My Institution',               # optional
    related_publications='DOI:10.1016/j.neuron.2016.12.011'   # optional
)
nwbfile

root pynwb.file.NWBFile at 0x1416388533512
Fields:
  experimenter: ['My Name']
  file_create_date: [datetime.datetime(2020, 5, 5, 9, 14, 53, 408752, tzinfo=tzlocal())]
  identifier: Mouse5_Day3
  institution: University of My Institution
  lab: My Lab Name
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: Head-fixed mouse on a wheel
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  timestamps_reference_time: 2018-04-25 02:30:03-07:00

## Subject information
Create a `Subject` object to store information about the experimental subject, such as age, species, genotype, sex, and a freeform description. And set `nwb.subject` to the `Subject` object.

<img src="images/subject.svg" width="200">

In [2]:
from pynwb.file import Subject

nwbfile.subject = Subject(
    subject_id='0001',
    age='P9M', 
    description='mouse 5',
    species='Mus musculus', 
    sex='M'
)

## SpatialSeries
Many types of data have special data types in NWB. To store the spatial position of a subject, we will use the `SpatialSeries` and `Position` classes. `SpatialSeries` is a subclass of `TimeSeries`. `TimeSeries` is a common base class for measurements sampled over time, and provides fields for data and time (regularly or irregularly sampled).

~~`SpatialSeries` is a subclass of `TimeSeries`. `TimeSeries` is a common base class for measurements sampled over time, and provides fields for data and time (regularly or irregularly sampled). The `Position` object is a special type of object called a `MultiContainerInterface`. It holds one or more `SpatialSeries` objects.~~

<img src="images/Position.svg">

Create a `SpatialSeries` object named `'SpatialSeries'` with some fake data.

In [3]:
import numpy as np
from pynwb.behavior import SpatialSeries

# create fake data with shape (100, 2)
# the first dimension should always represent time
position_data = np.array([np.linspace(0, 10, 100),
                          np.linspace(0, 8, 100)]).T
position_timestamps = np.linspace(0, 100) / 200

spatial_series_obj = SpatialSeries(
    name='SpatialSeries', 
    description='(x,y) position in open field',
    data=position_data,
    timestamps=position_timestamps,
    reference_frame='(0,0) is bottom left corner'
)

To help data analysis and visualization tools know that this `SpatialSeries` object represents the position of the animal, we will store the `SpatialSeries` object inside of a `Position` object.

~~Then, create a `Position` object which contains the `SpatialSeries` object you just created.~~

In [4]:
from pynwb.behavior import Position

position_obj = Position(spatial_series=spatial_series_obj)

NWB differentiates between raw, acquired data, which should never change, and processed data, which is the result of a data preprocessing algorithm and could change. Next, let's assume that the animal's position was computed from a video tracking algorithm, so it would be classified as processed data. Since processed data can be very diverse, let's store the animal's position data in a processing module that we create specifically for behavioral data.

~~Finally, create a processing module named `'behavior'` in the NWB file and add the `Position` object to the processing module.~~

In [5]:
behavior_module = nwbfile.create_processing_module(
    name='behavior',
    description='processed behavioral data'
)
behavior_module.add(position_obj)

Position pynwb.behavior.Position at 0x1416388532104
Fields:
  spatial_series: {
    SpatialSeries <class 'pynwb.behavior.SpatialSeries'>
  }

## Write to file

Now, let's try writing the NWB file that we have built so far.

In [6]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('ophys_tutorial.nwb', 'w') as io:
    io.write(nwbfile)

We can then read the file and print it to inspect its contents. You can then navigate to the `SpatialSeries` data that we created by referencing the names of the objects that contain it.

In [7]:
with NWBHDF5IO('ophys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()
    print(read_nwbfile)
    print(read_nwbfile.processing['behavior']['Position']['SpatialSeries'])

root pynwb.file.NWBFile at 0x1416680214088
Fields:
  experimenter: ['My Name']
  file_create_date: [datetime.datetime(2020, 5, 5, 9, 14, 53, 408752, tzinfo=tzoffset(None, -25200))]
  identifier: Mouse5_Day3
  institution: University of My Institution
  lab: My Lab Name
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: Head-fixed mouse on a wheel
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  subject: subject pynwb.file.Subject at 0x1416671626568
Fields:
  age: P9M
  description: mouse 5
  sex: M
  species: Mus musculus
  subject_id: 0001

  timestamps_reference_time: 2018-04-25 02:30:03-07:00

SpatialSeries pynwb.behavior.SpatialSeries at 0x1416671538952
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (100, 2), type "<f8">
  description: (x,y) position in open field
  interval: 1
  reference_frame: (0,0) is bottom l

## Trials

<img src="images/trials.svg">

`DynamicTable` objects are used to store tabular metadata throughout NWB, including electrodes and sorted units. They offer flexibility for tabular data by allowing required columns, optional columns, and custom columns.

Trials are stored in a `TimeIntervals` object which is a subclass of `DynamicTable`. Let's continue adding to our `NWBFile` by creating a new column for the trials table named `'correct'`, which will be a boolean array.

In [8]:
nwbfile.add_trial_column(name='correct', description='whether the trial was correct')
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Optical physiology
Optical physiology results are written in four steps:
1. Create imaging plane
2. Acquired two-photon images
3. Image segmentation
4. Fluorescence and dF/F responses

### Imaging Plane
First, you must create an `ImagingPlane` object, which will hold information about the area and method used to collect the optical imaging data. This requires creation of a `Device` object for the microscope and an `OpticalChannel` object. Then you can create an `ImagingPlane`.


<img src="images/imagingplane.svg">


In [9]:
from pynwb.device import Device
from pynwb.ophys import OpticalChannel

device = Device(name='Microscope001')
nwbfile.add_device(device)
optical_channel = OpticalChannel(
    name='OpticalChannel', 
    description='my_description', 
    emission_lambda=500.
)
imaging_plane = nwbfile.create_imaging_plane(
    name='ImagingPlane',
    optical_channel=optical_channel,
    imaging_rate=30.,
    description='a very interesting part of the brain',
    device=device,
    excitation_lambda=600.,
    indicator='GFP',
    location='V1',
    grid_spacing=[.01, .01],
    grid_spacing_unit='meters'
)

### TwoPhotonSeries
Now that you have your `ImagingPlane`, you can create a `TwoPhotonSeries` the class representing two photon imaging data. From here you have two options. The first option is to supply the image data to PyNWB, using the `data` argument. The other option is the provide a path the images. These two options have trade-offs, so it is worth spending time considering how you want to store this data.

<img src="images/twophotonseries.svg">


In [10]:
from pynwb.ophys import TwoPhotonSeries

# using internal data. This data will be stored inside the HDF5 file
image_series1 = TwoPhotonSeries(
    name='TwoPhotonSeries1',
    data=np.ones((1000,100,100)),
    imaging_plane=imaging_plane,
    rate=1.0,
    unit='normalized amplitude'
)
nwbfile.add_acquisition(image_series1)

# using external data
image_series2 = TwoPhotonSeries(
    name='TwoPhotonSeries2', 
    dimension=[2],
    external_file=['images.tiff'], 
    imaging_plane=imaging_plane,
    starting_frame=[0], 
    format='tiff', 
    starting_time=0.0, 
    rate=1.0
)
nwbfile.add_acquisition(image_series2)

### Plane Segmentation
Image segmentation stores the detected regions of interest in the `TwoPhotonSeries` data. `ImageSegmentation` allows you to have more than one segmentation by creating more `PlaneSegmentation` objects.

<img src="images/planesegmentation.svg" width="1000">

In [11]:
from pynwb.ophys import ImageSegmentation

img_seg = ImageSegmentation()
ps = img_seg.create_plane_segmentation(
    name='PlaneSegmentation',
    description='output from segmenting my favorite imaging plane',
    imaging_plane=imaging_plane,
    reference_images=image_series1  # optional
)
ophys_module = nwbfile.create_processing_module(
    name='ophys', 
    description='optical physiology processed data'
)
ophys_module.add(img_seg)

ImageSegmentation pynwb.ophys.ImageSegmentation at 0x1416680933192
Fields:
  plane_segmentations: {
    PlaneSegmentation <class 'pynwb.ophys.PlaneSegmentation'>
  }

### Regions Of Interest (ROIs)
#### Image masks
You may add ROIs using an image mask or using a pixel mask. An image mask is an array that is the same size as a single frame of the `TwoPhotonSeries`, and indicates where a single region of interest is. This image mask may be boolean or continuous between 0 and 1.

In [12]:
for _ in range(30):
    image_mask = np.zeros((100, 100))
    
    # randomly generate example image masks
    x = np.random.randint(0, 95)
    y = np.random.randint(0, 95)
    image_mask[x:x+5, y:y+5] = 1
    
    # add image mask to plane segmentation
    ps.add_roi(image_mask=image_mask)
    
# show one of the image masks
import matplotlib.pyplot as plt
plt.imshow(image_mask)

ModuleNotFoundError: No module named 'matplotlib'

#### Pixel masks
Alternatively, you could define ROIs using pixel mask, which defines each pixel individually with (x, y, val). All undefined pixels are assumed to be 0. But you need to decide and keep it consistent. You cannot add some by image_mask and some by pixel_mask.

In [13]:
ps2 = img_seg.create_plane_segmentation(
    name='PlaneSegmentation2',
    description='output from segmenting my favorite imaging plane',
    imaging_plane=imaging_plane,
    reference_images=image_series1  # optional
)

for _ in range(30):
    # randomly generate example image masks
    x = np.random.randint(0, 95)
    y = np.random.randint(0, 95)
    
    pixel_mask = []
    for ix in range(x, x+5):
        for iy in range(y, y+5):
            pixel_mask.append((ix, iy, 1))
    
    # add pixel mask to plane segmentation
    ps2.add_roi(pixel_mask=pixel_mask)

## Storing fluorescence measurements

Now that ROIs are stored, you can store fluorescence dF/F data for these regions of interest. This type of data is stored using the `RoiResponseSeries` class. You will not need to instantiate this class directly to create objects of this type, but it is worth noting that this is the class you will work with after you read data back in.

<img src="images/roiresponseseries.svg" width="1200">


First, create a data interface to store this data in

In [15]:
from pynwb.ophys import Fluorescence

fl = Fluorescence()
ophys_module.add(fl)

Fluorescence pynwb.ophys.Fluorescence at 0x1416680882568
Fields:

Because this data stores information about specific ROIs, you will need to provide a reference to the ROIs that you will be storing data for. This is done using a `DynamicTableRegion`, which can be created with `PlaneSegmentation.create_roi_table_region`.

In [16]:
rt_region = ps.create_roi_table_region(
    region=[0,1],
    description='the first of two ROIs'
)

Now that you have a `DynamicTableRegion`, you can create your an `RoiResponseSeries`.

In [17]:
roi_response_series = fl.create_roi_response_series(
    name='RoiResponseSeries',
    data=np.ones((50,2)),
    rois=rt_region,
    unit='lumens',
    rate=30.
)

## Write the file

In [18]:
with NWBHDF5IO('ophys_tutorial.nwb', 'w') as io:
    io.write(nwbfile)

## Reading NWB data
Data arrays are read passively from the file. Calling `TimeSeries.data` does not read the data values, but presents an `h5py` object that can be indexed to read data. Index this array just like a numpy array to read only a specific section of the array, or use the `[:]` operator to read the entire thing.

In [19]:
with NWBHDF5IO('ophys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()

    print(read_nwbfile.acquisition['TwoPhotonSeries1'])
    print(read_nwbfile.processing['ophys']['Fluorescence']['RoiResponseSeries'].data[:])

TwoPhotonSeries1 pynwb.ophys.TwoPhotonSeries at 0x1416678071816
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (1000, 100, 100), type "<f8">
  description: no description
  imaging_plane: ImagingPlane pynwb.ophys.ImagingPlane at 0x1416678071048
Fields:
  conversion: 1.0
  description: a very interesting part of the brain
  device: Microscope001 pynwb.device.Device at 0x1416677809864
  excitation_lambda: 600.0
  imaging_rate: 30.0
  indicator: GFP
  location: V1
  optical_channel: (
    OpticalChannel <class 'pynwb.ophys.OpticalChannel'>
  )
  unit: meters

  rate: 1.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: normalized amplitude

[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.

# Learn more!

## Python tutorials
### See our tutorials for more details about your data type:
* [Extracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py)
* [Calcium imaging](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#sphx-glr-tutorials-domain-ophys-py)
* [Intracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/icephys.html#sphx-glr-tutorials-domain-icephys-py)

### Check out other tutorials that teach advanced NWB topics:
* [Iterative data write](https://pynwb.readthedocs.io/en/stable/tutorials/general/iterative_write.html#sphx-glr-tutorials-general-iterative-write-py)
* [Extensions](https://pynwb.readthedocs.io/en/stable/tutorials/general/extensions.html#sphx-glr-tutorials-general-extensions-py)
* [Advanced HDF5 I/O](https://pynwb.readthedocs.io/en/stable/tutorials/general/advanced_hdf5_io.html#sphx-glr-tutorials-general-advanced-hdf5-io-py)


## MATLAB tutorials
* [Extracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ecephys.html)
* [Calcium imaging](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ophys.html)
* [Intracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/icephys.html)